# Vaccination Classification

![Vaccination Bottles](images/vaccine_bottles_shrunk.jpg)

**Author**: Carl Schneck<br>
**Program**: Data Science Flex<br>
**Phase 3 Project**

---

## Overview

Pandemics have occured throughout history, each time affecting a large portion of the population. The most recent case being the Covid-19 2020 outbreak which resulted in a prolonged change in peoples day to day life, a lack or resources, and ultimately many deaths all around the world. If possible governments will do their best to try and prevent any future outbreaks by observing and learning from past information.     

Therefore our stakeholder, a government agency, is trying to plan for future pandemic prevention and awareness by using the 2009 H1N1 pandemic as an example. They would like us to observe which features of a survey completed at the time appear hold the highest importance in people decisions to recieve the H1N1 vaccine. With this information they are hoping to be able to concentrate their efforts to provide vaccination information and flu prevention methods to a group that was otherwise more succeptible to contracting the virus. These efforts will be performed in hopes of increasing the vaccination rates and to help limit the spread of future viruses.

---

## Business Understanding

Our stakeholders goal is to help in the prevention of future pandemics by spreading vaccination and prevention information. To increase the value of their efforts they would like to target the population that are less inclined to receive the vaccination. The most successful outcome would be the prevention of a widespread outbreak resulting in a pandemic, but more realisticly an increase in vaccination awareness that results in a higher rate of people receiving vaccinations in case another outbreak occurs.

Therefore this projects requirements are to define the target audience who are less inclined to receive a vaccination. In order to do so a dataset on people who have both received and refrained from receiving vaccinations in the past is required. Available to us is a survey conducted for the 2009 H1N1 pandemic. The H1N1 outbreak was first detected in the United States and quickly spread across the rest of the world resuliting in between 151,000 and 575,000 deaths worldwide. Unlike prior strains of the H1N1 virus, people under 65 were more affected than the older population. Around 80 percent of the deaths assumed caused by this strain of H1N1 were people under the age of 65. Since this strain differed from previous strains the seasonal flu vaccinations didn't offer protection from the virus causing a late production of a vaccine that would be affective. An affective vaccination didn't get mass produced until after a second wave of the virus had come and gone in the United States <a href="#h1n1_cdc_article">[1]</a>.

Due to these factors this dataset may stray from a typical case, reason being: 

1. <strong>The late emergence of the mass produced vaccine.</strong> It wasn't until after the second outbreak had passed that it was available, possibly causing people to assume the worst was over and a vaccination wasn't required and lowering the number of vaccinations.

2. <strong>The age group most affected were people under 65.</strong> This isn't typical for outbreaks and may have caused the number of vaccinations in this age group to be inflated.

Though our dataset may not reflect these two points, they should be kept in mind while analyzing the results. As far as metrics to qualify our models performance, I believe accuracy and precision should be used. A result of having high precision will ensure that our feature importance will strongly relate to true positive entries. Though there should be a fine balance and recall shouldn't be completely forgone. While we don't want our predicted positives to contain many false positives, we would like to predict a good portion of our true positive entries. Approaching our problem by utilizing our metrics in this manner can assure that our stakeholder is targeting the correct audience.


---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
import sys
import pickle
import joblib
import python_code.my_functions as myfunc
import python_code.my_classes as mycl
import python_code.pres_figures as pfigs
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import (LabelEncoder, FunctionTransformer,
                                   OneHotEncoder, OrdinalEncoder,
                                   MinMaxScaler)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import (train_test_split, cross_val_score,
                                     GridSearchCV, cross_validate)
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import (BaseEstimator, TransformerMixin)
from sklearn.metrics import (accuracy_score, confusion_matrix,
                             classification_report, precision_score,
                             make_scorer)
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from itertools import combinations
from xgboost import XGBClassifier

%matplotlib inline

## Data Understanding

As mentioned earlier in the business understanding section, the data used for this analysis will be a 2009 survey conducted for the H1N1 outbreak. This survey was performed by the CDC in order to monitor and evaluate the flu vaccination efforts of adults and children in randomly selected US households. The questions asked of the participants dealt with their H1N1 vaccination status, flu-related behaviors, opinions about flu vaccine safety and effectivenss, recent respiratory illness, and pneumococcal vaccination status <a href="#About the National Immunization Survery">[2]</a>.

A detailed explanation of the features included in this survey can be found <a href="https://github.com/cschneck7/phase_3_project/blob/main/data/H1N1_and_Seasonal_Flu_Vaccines_Feature_Information.txt">here</a> or <a href="https://www.drivendata.org/competitions/66/flu-shot-learning/page/211/">here</a>.
<!--  link need to fitted to github repository when pushed -->

The following data from the survey can be found and downloaded <a href="https://www.drivendata.org/competitions/66/flu-shot-learning/data/">here</a> <a href="#Source Data Download">[3]</a>

In [ ]:
# Import survey data into dataframes
# The source dataset already had this split feature and target files
X = pd.read_csv('data/source_data/training_set_features.csv')
y = pd.read_csv('data/source_data/training_set_labels.csv')

The `training_set_labels.csv` file read to `y` contains two target variables, `h1n1_vaccine` and `seasonal_vaccine1`. For this project only the `h1n1_vaccine` target variable will be used.

In [ ]:
# Sets target variable
y = y.h1n1_vaccine

In [ ]:
# Checks distribution of target variable
print(y.value_counts())
print(y.value_counts(normalize=True))

Around 79% of those who were surveyed did not receive the vaccination. 

Let't take a look at the featuers.

In [ ]:
# preview of feature dataframe
X.head()

We don't need the `respondent_id` column, so it will be dropped.

In [ ]:
# drops respondent_id column
X.drop('respondent_id', axis=1, inplace=True)

# preview at feature column information
X.info()

This datasets contains 26,707 entries and 35 features. The features have been inputted as either float64 or object types, 23 and 12 columns of those types respectively. As mentioned in the <a href="https://github.com/cschneck7/phase_3_project/blob/main/data/H1N1_and_Seasonal_Flu_Vaccines_Feature_Information.txt">feature description</a> file, the float64 column types are either encoded or binary where Yes=1 and No=0. It can also be observed from the above information that there are many missing values.

In [ ]:
# Checks amount of Nan values in feature dataframe
missing_values = X.isna().sum()
missing_values

Almost all columns are missing values, and some are missing nearly half of their values missing. These columns may be dropped later while the others are filled in. 

In [ ]:
# Saving columns with many missing entries for later
many_missing = missing_values[missing_values > 12000]

Let's take a closer look at some of the features then appear to stand out as possible having an elevated importance to our target variable. As mentioned before `age_group` is an important metric in this study due to the unique trait of the H1N1 virus affecting different age groups than normal flu viruses. A few other features that may be interesting to have a further look at include:

- `doctor_recc_h1n1` - Whether H1N1 flu vaccine was recommended by their doctor
- `opinion_h1n1_vacc_effective` - Respondent's opinion about H1N1 vaccine effectiveness.
- `opinion_h1n1_risk` - Respondent's opinion about risk of getting sick with H1N1 flu without vaccine.
- `income_poverty` - Household annual income of respondent with respect to 2008 Census poverty thresholds.
- `race` - Race of respondent.

#### `age_group`

In [ ]:
# Finds unvaccinated group rates by age group
myfunc.vacc_rates(X.age_group, y)

As mentioned earler the H1N1 virus affected those under 65 more than those above that age, differing from the stereotypical flu. Looking at the grouping above those in the age group of 55-64 had the highest percentage of vaccinations, with 75.7 of people not vaccinated. Those younger than 55 were unvaccinated at a rate of around 80.5% while those above at 77%. I wouldn't say there is conclusive evidence from this chart that the uncharacteristic affect on the those under 65 had an affect on the vaccination rate in those age groups.

#### `doctor_recc_h1n1`

- H1N1 flu vaccine was recommended by their doctor.

No = 0 <br>
Yes = 1

In [ ]:
# Finds unvaccinated group rates by whether a doctor reccomended the H1N1 vaccination
myfunc.vacc_rates(X.doctor_recc_h1n1, y)

There's a clear importance in regards to vaccination status depending on doctor reccomendation. Of those that were recommended the vaccination only 46.7% of them didn't get the vaccine, while 86.3% of those who weren't recommended the vaccine didn't get the vaccine. 

#### `opinion_h1n1_vacc_effective`

- Respondent's opinion about H1N1 vaccine effectiveness

1 = Not at all Effective<br>
2 = Not very Effective<br>
3 = Don't Know<br>
4 = Somewhat Effective<br>
5 = Very Effective

In [ ]:
# Finds unvaccinated group rates by opinion of H1N1 vaccination effectiveness
myfunc.vacc_rates(X.opinion_h1n1_vacc_effective, y)

There is a clear correlation with the opinion of vaccine effectiveness and vaccination rate. As the opinion of effectiveness got higher the the vaccination rate drastically increased.

#### `opinion_h1n1_risk`

- Respondent's opinion about risk of getting sick with H1N1 flu without vaccine

1 = Very Low<br>
2 = Somewhat Low<br>
3 = Don't Know<br>
4 = Somewhat High<br>
5 = Very High

In [ ]:
# Finds unvaccinated group rates by opinion of H1N1 risk 
myfunc.vacc_rates(X.opinion_h1n1_risk, y)

Similarly to `opinion_h1n1_vacc_effective`, the rate of vaccinations increased as the opinion of risk got higher. Though there appears to be a much smaller group who consider the H1N1 flu's risk to be high in comparison to the group who consider the vaccine to be effective.

#### `income_poverty`

- Household annual income of respondent with respect to 2008 Census poverty thresholds

In [ ]:
# Finds unvaccinated group rates by household income
myfunc.vacc_rates(X.income_poverty, y)

The rate of vaccination appears to be correlated to household income, with the percentage increasing with household income. This could possibly be due to lower income households not having access to health care.

In [ ]:
# Quick rates on those without health insurance split by household income groupings
myfunc.vacc_rates(X.income_poverty, X.health_insurance)

The chart above shows those who don't have insurance in those groupings. There is a clear correlation between income and percentage of people without insurance with 32.6% of those below poverty without insurance, while only 3% those with above \$75k household income are without insurance.

#### `race`

In [ ]:
# Finds unvaccinated group rates by respondent's race
myfunc.vacc_rates(X.race, y)

The vaccination rates appear to be similar for all races besides `black` where the vaccination rate is about 6% lower.

### Feature Correlation Using Chi2

Now lets check correlation between our target variable and features. We will be using a Chi2 test, with alpha=.05 and our null hypothesis being that there is no relationship between our feature and target variable.

Hypothesis:<br><br>
&emsp;&emsp;<strong>H<sub>0</sub></strong>: No relationship between the feature and target variable
<br>
&emsp;&emsp;<strong>H<sub>1</sub></strong>: There is a relationship between the feature and target variable   

In [ ]:
myfunc.ordered_chi2(X, y)

The above dataframe tells us that all but two features have failed the null hypothesis (H<sub>0</sub>) and are significant to our target variable. The two features that ended up being insignificant are `household_children` and `census_msa`.

The top six features in terms of significance were:

- `opinion_seas_risk`
- `opinion_h1n1_risk`
- `opinion_h1n1_vacc_effective`
- `doctor_recc_h1n1`
- `doctor_recc_seasonal`
- `opinion_seas_vacc_effective`

A theme of the top six is that there are basically three topics, asked each about the H1N1 flu and Seasonal flu. The initial assumption is that these pairs may be highly correlated. Let's confirm this by checking the correlation of all features using the Chi2 test again with an alpha of .05. The hypothesis are similar to the previous test between features and target variable.

Hypothesis:<br><br>
&emsp;&emsp;<strong>H<sub>0</sub></strong>: No relationship between the two features
<br>
&emsp;&emsp;<strong>H<sub>1</sub></strong>: There is a relationship between the two features 

In [ ]:
alpha=.05
chi2_df = myfunc.ordered_chi2(X, alpha=alpha)
chi2_df

According to the DataFrame above, many features are correlated. There are a total of 573 instances of high correlation between features. Considering we have many missing values to deal with, the high correlation between certain features may help us fill in some missing data. Let's check if our assumption about the questions related to both the H1N1 and Seasonal flu having high multicollinearty is true.

In [ ]:
similar_features = [['doctor_recc_h1n1', 'doctor_recc_seasonal'],
                    ['opinion_h1n1_vacc_effective', 'opinion_seas_vacc_effective'],
                    ['opinion_h1n1_risk', 'opinion_seas_risk'],
                    ['opinion_h1n1_sick_from_vacc', 'opinion_seas_sick_from_vacc']]

for pair in similar_features:
    pvalue = "{:e}".format(float(chi2_df.loc[((chi2_df.var1 == pair[0]) & (chi2_df.var2 == pair[1])) |
                ((chi2_df.var2 == pair[0]) & (chi2_df.var1 == pair[1])), 'Pvalue']))
    print(f'({pair[0]}, {pair[1]}) : {pvalue}')

As seen above our assumptions were correct, for all cases we can reject the null hypothesis of the features having no relationship.

## Data Preparation

Now that we have a better understanding of our data, let's prepare it for our models. 

We'll start with dropping columns and row containing many missing entries. We already found the columns that fit this criteria earlier. While we are performing this all duplicate entries will be dropped.

In [ ]:
X_prep = X.copy()

X_prep.drop(many_missing.index, axis=1, inplace=True)

# Drops duplicate entries if any exist
df_all = pd.concat([X_prep, y], axis=1).drop_duplicates()
X_prep = df_all.drop(y.name, axis=1)
y_prep = df_all[y.name]

print(f'Original number of entries: {X.shape[0]}')
print(f'Number of entries after dropped Duplicates: {X_prep.shape[0]}')

There weren't any duplicate entries.

Before we start editing more values and rows lets create a train-test split to prevent data leakage.

In [ ]:
# Creates training and test splits to prevent data leakage
X_train, X_test, y_train, y_test = train_test_split(X_prep, y_prep, test_size=.25, random_state=42)

Now lets check how many rows are missing many values.

In [ ]:
# Find row missing nan information
myfunc.missing_row_entries(X_train)

Our current features dataframe has 32 features after dropping those four columns. That means around 353 entries are missing about a third (11 values) of their features information. Since these entries are missing most of their information lets drop them from the dataset.

In [ ]:
# Creates custom fuction transfromer
Many_Nans_Row_Drop_FT = FunctionTransformer(myfunc.drop_rows_by_nans, kw_args = {'y': 'h1n1_vaccine',
                                                                          'nan_threshold': 11})

In [ ]:
# Creates combined set of both features and target to be transformed
train_df = pd.concat([X_train, y_train], axis=1).copy()

# Transforms training set
X_train_mod, y_train_mod = Many_Nans_Row_Drop_FT.fit_transform(train_df)

# Checks change in shape to ensure proper amount of entries were dropped
X_train.shape[0] - X_train_mod.shape[0]

Our object columns now need to be encoded for later use in our models.

In [ ]:
# Creates dataframe of object types
X_train_mod_obj = X_train_mod.select_dtypes(include='object').copy()

# Creates dataframe of numerical features
X_train_mod_num = X_train_mod.select_dtypes(exclude='object').copy()

# Initialized ordinal encoder object
oe = OrdinalEncoder()

# Creates dataframe of ordinal encoded data
X_train_mod_obj = pd.DataFrame(data = oe.fit_transform(X_train_mod_obj),
                                columns=X_train_mod_obj.columns,
                                index=X_train_mod_obj.index)

# Checks newly encoded dataframe
X_train_mod_obj.head()

In [ ]:
X_train_enc = pd.concat([X_train_mod_obj, X_train_mod_num], axis=1)

Let's now try and fill in the rest of our missing values. Earlier we noticed that many variables had high colinearity with each other, we can use this to help us fill in for missing values. Our first attempt at filling the values will be through classification using the other features.

Issue with this process is the existence of missing values, many models in scikit-learn libraries cannot handle Nan values. Therefore random imputation will be used to fill in the feature information. The values will be randomly selected using existing values and their rate of appearances in the features existing values. After that is completed an iterative Decision Tree Classifier will be used to fill the randomly imputed values with predicted values.

This process was modified from Shashanka Subrahmanya's process for a regression model in his article <a href="https://www.kaggle.com/code/shashankasubrahmanya/missing-data-imputation-using-regression">Missing Data Imputation using Regression</a> <a href='#Missing Data Imputation using Regression'>[4]</a>. His process utilized linear regression and didn't create custom transfomers. For the purpose of this project I have modified it to fit my needs.

This next cell will create a RandomImputer object, fit and transform it to our training feature dataset, and then check if all the missing values have been filled.

In [ ]:
# Creates class object
rand_imp = mycl.RandomImputer()

# Fits and transforms dataset
X_train_imp = rand_imp.fit_transform(X_train_enc)

# checks if all columns besides original columns with missing data are filled in
X_train_imp.drop(rand_imp.missing_columns, axis=1).isna().sum()

Now that each feature has been filled with randomly imputed data, it's time to apply a regression models to predict the values for missing data from the other features data instead of randomly imputing the data.

This requires an iterative process for each feature that was originally missing data. Since each column is categorical they will need to be OHE for each iteration and the target variable (column missing data) will need to be treated as a multiclass classification problem. Also since the features contain various amount of information, I would like to iterate through the columns filling by number of missing values. I am not sure if filling in the columns with little data missing first, or a lot of missing data is better. By filling in the columns with less missing data first, it could help achieve more accurate information to fill the columns with many missing entries. On the other hand filling in the columns with many missing values, will take out more randomness for future iterations, by filling in more random values with predicted values from classification. To start I believe I will try the method of handling columns with many missing entries first. Also since many columns and iterations are needed, Decision Trees will be used to predict the missing values.

The IterativeClassification class uses a OneHotEncoding transformer to help fit the data, therefore the categorical and numerical columns must be supplied.

The features to be considered categorical will be those with more than 2 unique values, aren't opinion based, or age. The opinion based features like `h1n1_concern` have values from 1-5 that are in order of magnitude.  

In [ ]:
# Finds all features that are opinion based with entries ranked from 1-5
opinion_cols = [col for col in X_train_mod.columns if ('opinion' in col)] + ['h1n1_concern', 'h1n1_knowledge']

# Finds features that have more than 2 unique values
many_unique_cols = X_train_mod.nunique() > 2
many_unique = many_unique_cols.index[many_unique_cols]

# Subtracts opinion features from list with features with greater than 2 unique values
# to get categorical columns for one hot encoding
cat_cols = list(set(many_unique) - set(opinion_cols) - {'age_group'})

# Previews left over categorical columns
cat_cols

In [ ]:
# Creates list of numerical features
num_cols = list(set(X_train_mod.columns) - set(cat_cols))

In the next cell the IterativeClassification class object is created, fit and the accuracy scores are checked. The accuracy score was checked with each iteration with the first entry being the first iteration.

In [ ]:
# Creates IterativeClassification class object with max_depth = 5
iter_class = mycl.IterativeClassification(max_depth=5, cat_cols=cat_cols, num_cols=num_cols)

# Fits object to training data
iter_class.fit(X_train_imp)

# Checks accuracy model by checking predictions against existing values
iter_class.fit_accuracy_scores

Though the decision tree classifier didn't work great for all columns, it can definently be said that it performed better than random guessing. Let's transform our training set to fill in the predicted values.

In [ ]:
# transforms dataset to fill in missing values
X_train_trans = iter_class.transform(X_train_imp)

# print first 5 entries of transformed dataset
X_train_trans.head()

In [ ]:
iter_class.pred_features

In [ ]:
# # Saves X_train_trans to pickle file for ease to reload
# with open('data/temp_pickle_files/X_train_trans.pickle', 'wb') as f:
#     pickle.dump(X_train_trans, f)

# Opens X_train_trans pickle file
with open('data/temp_pickle_files/X_train_trans.pickle', 'rb') as f:
    X_train_trans = pickle.load(f)

Our dataset now has no missing values. The features with `Det_` starting their names, are those that have been transformed to have their missing values filled in.

Now that we've filled in our missing values, let's drop some features that have clear multicollinearity, and poor correlation with our target variable. 

The features with high multicollinearity that will be dropped were those about the seasonal flu that had a similar question related to the H1N1 flu. These features included:

- `doctor_recc_seasonal`
- `opinion_seas_vacc_effective`
- `opinion_seas_risk`
- `opinion_seas_sick_from_vacc`

Also found earlier those with poor relationships to our target variable:

- `household_children`
- `and census_msa`

In [ ]:
# original column names that need to be dropped
orig_cols = ['doctor_recc_seasonal', 
             'opinion_seas_vacc_effective', 
             'opinion_seas_risk', 
             'opinion_seas_sick_from_vacc', 
             'household_children', 
             'census_msa']

# finds current column names in transformed dataframe
drop_cols = [col for col in X_train_trans.columns if any(orig_col in col for orig_col in orig_cols)]

# Drops columns with poor relationship and multicollinearity
X_train_mod2 = X_train_trans.drop(drop_cols, axis=1).copy()

Now that we have no missing values, and have performed some feature selection, we need to One Hot Encode all our categorical columns.

In [ ]:
# Gets feature names of those to be one hot encoded
ohe_columns = [col for col in X_train_mod2.columns if any(cat_col in col for cat_col in cat_cols)]

# creates column transformer object with OHE
ct_ohe = ColumnTransformer([('ohe', OneHotEncoder(sparse=False), ohe_columns)],
                           remainder='passthrough')

# fits and transforms column transformer to X_train_mod2
X_train_ohe_array = ct_ohe.fit_transform(X_train_mod2)

# Get new column names
ct_col_names = ct_ohe.get_feature_names_out()

# Remove transformer name and decimals from columns names
ct_col_names = [col.replace('ohe__', '').replace('remainder__', '').replace('.0', '') for col in ct_col_names]

# Creates dataframe with newly encoded data
X_train_ohe = pd.DataFrame(X_train_ohe_array,
                           columns = ct_col_names,
                           index=X_train_mod2.index)

# Previews first few entries of data
X_train_ohe.head(3)

Our next step is scaling our data. Most of our features have ranges of [0,1], but others are ranged from 1 to 5. This may cause bias in our models, so we will use a MinMaxScaler to adjust our scales.

In [ ]:
# Creates scaler object
scaler = MinMaxScaler()

# fits scaler object and transforms X_train_ohe and creates a dataframe
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_ohe),
                              columns=X_train_ohe.columns,
                              index=X_train_ohe.index)

# previews dataframe
X_train_scaled.head(3)

With all that our training features are ready to be used in a model.

Let's now bring our test sets up to date with the transformations. The transformations performed on our training set included:

1. Dropping rows with many values
2. Encoded our object features
3. Random Imputation for missing values
4. Using a Decision Tree Classifier to help better predict those missing values
5. Drop unnecessary features
6. One Hot Encode all categorical features
7. Scales Data

In [ ]:
# 1. Combines test features and target for first transformation
test_df = pd.concat([X_test, y_test], axis=1)

# 2. Drops rows with many missing entries
X_test_mod, y_test_mod = Many_Nans_Row_Drop_FT.transform(test_df)


# 3. Encoding
# Creates dataframe of object types
X_test_mod_obj = X_test_mod.select_dtypes(include='object').copy()
# Creates dataframe of numerical features
X_test_mod_num = X_test_mod.select_dtypes(exclude='object').copy()

# Creates dataframe of ordinal encoded data
X_test_mod_obj = pd.DataFrame(data = oe.transform(X_test_mod_obj),
                                columns=X_test_mod_obj.columns,
                                index=X_test_mod_obj.index)

# Creates ordinal encoded set
X_test_enc = pd.concat([X_test_mod_obj, X_test_mod_num], axis=1)

# 4. Random Imputation for missing values
X_test_imp = rand_imp.transform(X_test_enc)

# 5. Regression to predict randomly imputed values
X_test_trans = iter_class.transform(X_test_imp)

# 5. Drops unnecessary features
X_test_mod2 = X_test_trans.drop(drop_cols, axis=1).copy()

# 6. One hot encodes categorical features
X_test_ohe = pd.DataFrame(ct_ohe.transform(X_test_mod2),
                          columns = ct_col_names,
                          index=X_test_mod2.index)

# 7. MinMaxScales data
X_test_scaled = pd.DataFrame(scaler.transform(X_test_ohe),
                             columns=X_test_ohe.columns,
                             index=X_test_ohe.index)

# Checks shape to confirm size matches with training set
X_test_scaled.shape

## Modeling

### Base Model

Now that we have working datasets, let's begin the modeling process. Let's first create a basic model using a Decision Tree Classifier. We will cross validate the training set to prevent data leakage.

For the scoring method to work properly, the target variables values have to be flipped since we're interested in predicting those who are not vaccinated.

Therefore the values will be as followed, the old values are shown for reference:

&emsp;Old: (Vaccinated = 1, Not-Vaccinated=0)<br>
&emsp;New: (Vaccinated = 0, Not-Vaccinated=1)

In [ ]:
def binary_flip(ds):
    '''
    Takes in a pd.Series with [0,1] values and flips them.
    ex. 0 to 1, 1 to 0
    '''
    return (ds+1).mod(2)

In [ ]:
# Flips values of labels
y_train_flipped = binary_flip(y_train_mod)

In [ ]:
dt = DecisionTreeClassifier(max_depth=5, random_state=42)

# Sets scores to be calculated
scores = {'prec': 'precision',
          'acc': 'accuracy',
          'recall': 'recall',
          'ROC': 'roc_auc'}

cv_base_score = cross_validate(estimator=dt,
                               X=X_train_scaled,
                               y=y_train_flipped,
                               scoring=scores,
                               return_train_score=True)

cv_base_scores_df = pd.DataFrame(cv_base_score)
mean_df = pd.DataFrame(cv_base_scores_df.mean(), columns=['mean'])

cv_base_scores_df = pd.concat([cv_base_scores_df.T, mean_df], axis=1)
cv_base_scores_df

Our base model appears to have performed pretty well but may be slightly over fit. There's a bit of variance between the folds. It's accuracy of 82.2% is around 4% better than if we guessed with the 78.7% majority that didn't receive the vaccine.

Currently our base model has a precision of .855, meaning out of all our non-vaccinated predictions, 85.5% of them were correct. Our recall was .938 meaning we captured 93.3% of entries that weren't vaccinated. Though as mentioned before we want to concentrate our model qualification off of our precision, so let's try to improve this with future models.

### Model 2

For this model we will introduce an iterative approach through GridSearchCV. GridSearchCV helps by searching through combinations of hyperparameters for a given estimator. We will continue to use a DecisionTreeClassifier as our estimator at this step.

In [ ]:
# Creates DecisionTreeClassifier parameter grid
dt_par_grid = {'criterion': ['gini', 'entropy'],
            'max_depth': [4, 5, 6, 7, 8, 9, 10],
            'min_samples_split': [.01, .025, .05, .1]}

# Creates GridSearchCV object
dt_clf_grid = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   param_grid=dt_par_grid,
                   scoring=scores,
                   return_train_score = True,
                   refit='prec')

In [ ]:
# fits training values to grid search object
# dt_clf_grid.fit(X_train_scaled, y_train_flipped);

In [ ]:
# # Saves dt_clf_grid model to prevent long run times
# with open('data/models/dt_clf_grid.plk', 'wb') as f:
#     joblib.dump(dt_clf_grid, f)

# Loads dt_clf_grid model to prevent long run times
with open('data/models/dt_clf_grid.plk', 'rb') as f:
    dt_clf_grid = joblib.load(f)

In [ ]:
dt_clf_grid.best_score_

The best model from this grid search had a precision of around .86, only around .5% better than our base model. Let's check the parameters.

In [ ]:
# Creates DataFrame of results
dt_clf_df = pd.DataFrame(dt_clf_grid.cv_results_)

In [ ]:
# Finds the model with the highest ranked precision score
list(dt_clf_df.loc[dt_clf_df.rank_test_prec==1, 'params'])

It appears that multiple combinations of hyperparameters result in the same precision score. Observing the hyperparameter above, it seems like our models performed best when limited to a `min_sample_split` of around 5%. This parameter specifies the smallest leaf node sample size, in this case a leaf node can't have less than 5% of the samples. In the cases above all the models probably only had a depth of 4, resulting in the same score precision score. A higher rate of false positives must be getting predicted for the models able to split into smaller leaf nodes.

Let's observe if these models were overfit by taking a look at the individual folds.

In [ ]:
# Creates dataframe of best scorer row and transposes it to a column
best_scorer_dt = dt_clf_df.loc[dt_clf_df.rank_test_prec==1, :].T

In [ ]:
# displays DataFrame containing fold scores, mean score 
# and std_dev for both training and test set
myfunc.fold_scores(best_scorer_dt, 'prec')

Our train and test sets performed pretty closely with eachother and our folds variance wasn't large meaning we aren't over fitting. Let's check the rankings and other metrics for these models.

In [ ]:
# List rankings and means of all scoring metrics
myfunc.rankings(best_scorer_dt)

This models accuracy of around 82.3% and recall of 92.8% dropped slightly from our base models of 82.3% and 93.3% respectively. While our model performed the best in precision, this wasn't the case for all the other metrics. Let's create a figure observing the metrics in order of precision ranking.

In [ ]:
dt_clf_df.sort_values(by='rank_test_prec', inplace=True)

score_vars = [col for col in dt_clf_df.columns if ('mean_test' in col)]

fig, ax = plt.subplots(figsize=(6,6));

for i, col in enumerate(score_vars):
    ax.plot(dt_clf_df.rank_test_prec, dt_clf_df[col], label=col)

ax.set_xlabel('Precision Score Ranking');
ax.set_ylabel('Mean Score');
ax.set_title('Mean Scores of Iterations by Order of Precision Score Ranking');
ax.legend();

In [ ]:
# Saves current best models scores and hyperparameters for later reference
best_model_scores = myfunc.rankings(best_scorer_dt).loc[:, 2]
best_model_scores.rename('best_model', inplace=True)
best_model_params = dt_clf_grid.best_params_

As our precision drops our recall increases while our accuracy stays around the same. Even though we can see slight changes in the performance over these models, the scores only vary around one to two percentage points.

So far this is our best model, but it wasn't much of an improvement over our base model. Therefore let's try a different modeling method.

### Model 3

For this model will try using gradient boosting with XGBoost. Boosting is an iterative method that starts by building a weak model, and learning from it. This process is repeated until a stopping condition is met and our model is formed.

In [ ]:
# Instantiate XGBClassifier
xgb_clf= XGBClassifier(random_state=42)

# Creates parameter grid for search
param_grid = {
    'learning_rate': [0.1, 0.2, .3],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [.5, 1, 2],
    'subsample': [0.5, 0.7],
    'n_estimators': [100],
}

# creates gridsearch object using XGBoost as estimator
xgb_clf_grid = GridSearchCV(xgb_clf, param_grid,
                            scoring=scores,
                            return_train_score = True,
                            refit='prec')

# fits xgb_clf to training data
# xgb_clf_grid.fit(X_train_scaled, y_train_flipped)

In [ ]:
# # Saves xgb_clf_grid model to prevent long run times
# with open('data/models/xgb_clf_grid.plk', 'wb') as f:
#     joblib.dump(xgb_clf_grid, f)

# Loads xgb_clf_grid model to prevent long run times
with open('data/models/xgb_clf_grid.plk', 'rb') as f:
    xgb_clf_grid = joblib.load(f)

In [ ]:
xgb_clf_grid.best_score_

Our new models performance of 86% is almost identical to our current best model. Lets check to see if this model is overfitting and how well it performed with the other scores

In [ ]:
# Creates DataFrame of results
clf_boost_df = pd.DataFrame(xgb_clf_grid.cv_results_)

# Creates dataframe of best scorer row and trasposes it to a column
best_scorer_booster = clf_boost_df.loc[clf_boost_df.rank_test_prec==1, :].T

# displays DataFrame containing fold scores, mean score 
# and std_dev for both training and test set
myfunc.fold_scores(best_scorer_booster, 'prec')

It appears that our model migh be slightly overfit with the training score being around 2% higher than our test scores. Let's take a look at how this models other scores compare to our current best model.

In [ ]:
# List rankings and means of all scoring metrics, and compares to current best model
pd.concat([myfunc.rankings(best_scorer_booster), best_model_scores], axis=1)

This model improved on the metrics from the current best model, but performs poorly in all metrics besides precision compared to the other models in this grid search. It also appears to be overfit with it's training metrics being atleast 2% better than it's test metrics. Since we are also interested in high accuracy let's check how the model with the highest accuracy performed.

In [ ]:
# Creates dataframe of best scorer row by accuracy and trasposes it to a column
best_acc_booster = clf_boost_df.loc[clf_boost_df.rank_test_acc==1, :].T

# displays DataFrame containing fold scores, mean score 
# and std_dev for both training and test set
myfunc.fold_scores(best_acc_booster, 'prec')

This models test precision score is almost identical to the top model by precision in this grid search, while also appearing to be better fit by being only .5% lower than the training precision score. The variance of the folds is also much smaller than the previous model. Let's this models other scores and how it compares to our current best model.

In [ ]:
# List rankings and means of all scoring metrics, and compares to current best model
pd.concat([myfunc.rankings(best_acc_booster), best_model_scores], axis=1)

This models test scores are much closer to the training scores than the top model in terms of precision. It also performed slightly better in terms of recall while being almost identical in terms of precision. It also out performs our current est model in terms of accuracy and recall increasing the scores from 82.3% to 83.4% and 92.8% to 94.2% respectively. Thus our best model will be updated to this one.

In [ ]:
# Saves current best models scores and hyperparameters for later reference
best_model_scores = myfunc.rankings(best_acc_booster)
best_model_scores.rename(columns={best_model_scores.columns[0]: 'best_model'}, inplace=True)
best_model_params = best_acc_booster.loc['params', best_acc_booster.columns[0]]

Since we still haven't seen much improvement on our base model, let's move on with another model.


## Model 4 - Random Forest

Random forest models combine multiple decision trees to decide the output. The output with the most decision trees predicting it is the one that is chosen overall for that entry. 

In [ ]:
rf_clf = RandomForestClassifier(random_state=42, bootstrap=True)

param_grid_rf = {'n_estimators': (50, 100, 200, 400),
                 'criterion': ('gini', 'entropy', 'log_loss'),
                 'max_depth': (3, 4, 5, 6),
                 'max_samples': (.3, .5, .7)}

# creates gridsearch object using XGBoost as estimator
rf_clf_grid = GridSearchCV(rf_clf, 
                           param_grid_rf,
                           scoring=scores,
                           return_train_score = True,
                           refit='prec')

# fits RandomForest Grid Search to training data
# rf_clf_grid.fit(X_train_ohe, y_train_flipped)

In [ ]:
# # Saves rf_clf_grid model to prevent long run times
# with open('data/models/rf_clf_grid.plk', 'wb') as f:
#     joblib.dump(rf_clf_grid, f)

# Loads rf_clf_grid model to prevent long run times
with open('data/models/rf_clf_grid.plk', 'rb') as f:
    rf_clf_grid = joblib.load(f)

In [ ]:
rf_clf_grid.best_score_

Our random forest model performed worse than our previous best models with a precision score of 82%, down from around 86%. Let's take a look at the parameters and folds of our best random forest model.

In [ ]:
rf_clf_grid.best_params_

In [ ]:
# Creates DataFrame of results
rf_clf_df = pd.DataFrame(rf_clf_grid.cv_results_)

# Creates dataframe of best scorer row and trasposes it to a column
best_scorer_rf = rf_clf_df.loc[rf_clf_df.rank_test_prec==1, :].T

# displays DataFrame containing fold scores, mean score 
# and std_dev for both training and test set
myfunc.fold_scores(best_scorer_rf, 'prec')

Our random forest model isn't overfitting at the set hyperparameters. It performs best with a max depth of 6, with 200 estimators and while sampling half of our entries each time. Let's see how this model performed on the other scoring metrics compared against the current best model

In [ ]:
# List rankings and means of all scoring metrics, and compares to current best model
pd.concat([myfunc.rankings(best_scorer_rf), best_model_scores], axis=1)

While this model performed a few percentage points worse in precision than our best model so far, it has the highest recall score at .98, meaning it captures around 98% of our non-vaccinated entries. Our concentration is on the precision score of our model, since this one performed poorly in that regard it will not replace our best model.

So far out models have been focused around types of decision trees, for our next model let's try something different.

## Model 5

For the next model we will try a Logistic Regression model. Similarly to linear regression, logistic regression uses a loss function to help predict the target. 

In [ ]:
lr_clf = LogisticRegression(random_state=42)

param_grid_lr = {'penalty': ('l1', 'l2'),
                 'C': (.1, 1, 10),
                 'solver': ['liblinear'],
                 'max_iter': [200]}

# creates gridsearch object using XGBoost as estimator
lr_clf_grid = GridSearchCV(lr_clf, 
                           param_grid_lr,
                           scoring=scores,
                           return_train_score = True,
                           refit='prec')

# fits RandomForest Grid Search to training data
# lr_clf_grid.fit(X_train_ohe, y_train_flipped)

In [ ]:
# Saves lr_clf_grid model to prevent long run times
# with open('data/models/lr_clf_grid.plk', 'wb') as f:
#     joblib.dump(lr_clf_grid, f)

# # Loads lr_clf_grid model to prevent long run times
with open('data/models/lr_clf_grid.plk', 'rb') as f:
    lr_clf_grid = joblib.load(f)

In [ ]:
lr_clf_grid.best_score_

Our logistic regression model performed a little bit worse than our best model at 85.2% precision compared to 86%. Let's check its other scores.

In [ ]:
# Creates DataFrame of results
lr_clf_df = pd.DataFrame(lr_clf_grid.cv_results_)

# Creates dataframe of best scorer row and trasposes it to a column
best_scorer_lr = lr_clf_df.loc[lr_clf_df.rank_test_prec==1, :].T

# List rankings and means of all scoring metrics, and compares to current best model
pd.concat([myfunc.rankings(best_scorer_lr), best_model_scores], axis=1)

Comparing the scoring metrics to our best model, the logistic regression model performed worse except for recall. Therefore our best model will stay the same.

---

## Evaluation

### Final Model

Our best model is the XGBoosted model with the following parameters.

In [ ]:
best_model_params

Let's train this model on our full training set and check how it performs on our test set.

In [ ]:
# Creates XGBoost Classifier object with best model params
final_model = XGBClassifier(learning_rate=.2,
                            max_depth=3,
                            min_child_weight=.5,
                            n_estimators=100,
                            subsample=.7,
                            random_state=42)

# fits classifier to training dataset
final_model.fit(X_train_scaled, y_train_flipped);

In [ ]:
# Saves  final_model
with open('data/models/final_model.plk', 'wb') as f:
    joblib.dump(final_model, f)

# # Loads final_modell to prevent long run times
# with open('data/models/final_model.plk', 'rb') as f:
#     final_model =joblib.load(f)

In [ ]:
# Finds predicted values for train and test sets
y_train_pred = final_model.predict(X_train_scaled)
y_test_pred = final_model.predict(X_test_scaled)

In [ ]:
# Prints out confusion matrix and metric report for training set
myfunc.quick_metrics(y_train_flipped, y_train_pred)

In [ ]:
# Prints out confusion matrix and metric report for test set
y_test_flipped = binary_flip(y_test_mod)
myfunc.quick_metrics(y_test_flipped, y_test_pred)

Our training and test set scores match up pretty well so we know we aren't overfitting. This model also has the highest precision score out of all the models we have created, therefore fitting our goals. Testing for precision ensures that we have a high accuracy for those predicted to be unvaccinated. This is good for our business problem because we want to try and target the features whom strongly affect vaccination status. Let's now check which features hold the most importance in this model.

In [ ]:
feat_imp = list(zip(final_model.feature_importances_, X_train_scaled.columns))
feat_imp.sort(reverse=True)
feat_imp[:10]

Above shows the top 10 features in order of importance to our model. The top feature by a wide margin was `Det_doctor_recc_h1n1`. This agrees with the importance of a doctor recommendation that we saw in our data exploration stage, with a 53.3% vaccination rate in those who were recommended the vaccine compared to 13.6% of those who were not. Also agreeing with our earlier analysis is the importance of `Det_opinion_h1n1_vacc_effective` and `Det_opinion_h1n1_risk`. As the respondent's opinions on H1N1 risk and vaccination effectiveness increased so did the vaccination rate. 

The final model had a precision of 86% meaning of those predicted as unvaccinated 86% of them were correct. The total accuracy was also 84%, not much lower than our precision. According to the model, having a doctor recommend the vaccination had the greatest importance in one's decision. Followed by the respondent's opinions on the H1N1 vaccination effectiveness and H1N1 risk. Even though the vaccination rate was high in these groups the groups were much smaller then their counterparts. The vaccination rates for the groups who were recommended the vaccine, believed the vaccine was very effective or believed it was a high risk had vaccination rates between 40% - 54%. Though they only contained between 6% to 27% of the total amount of respondents.

---

## Conclusion

Pandemics are an ongoing concern for the entire population, with proper research our stakeholder is hoping to help in their prevention as well as increasing vaccination awareness. The goal was to find which features stand out the most in one's decision making process to receive a vaccination. The data available to us was a 2009 survey performed after the H1N1 pandemic. Our focus was to create a model with a high precision to ensure the correct target audience is located. Thus the best model in regards to precision will educate us on features that had the most weight in one's choice to get the vaccination. 

Observing the features that hold the most impact on the respondent's vaccination status, the conclusion that general practitioners hold the most weight in one's decision to be vaccinated can be obtained. As stated 53.3% of people with a doctors recommendation received the vaccine, compared to 13.6% of people who were not. Also it may be assumed that people receive information on vaccine effectiveness and virus risk from their personal health care physicians. Therefore I recommend reaching out to general practicioners with general information about vaccines or viruses to then transfer to their patients. This could possibly be in the form of pamphlets, posters or other sources of media. It should also be shown to the general practicioners how important their recommendation is in the outcome of their patient's decision. Only 22% of the respondant's in the survey were recommended the vaccine, yet more than half of those who were vaccinated were a part of that group.

---

## Future Improvements
<br>
<strong>Take a deeper look into the features that showed great importance.</strong> This includes finding subgroups in those features to help narrow down the target audience. For example we found out that those who believed the vaccination was effective had a much lower chance of being vaccinated, but could this be narrowed down to find the age group or other features that filled these sub groups more than others in order to find another group to target.<br><br>
<strong>Fill out missing data about health insurance.</strong> A lot of entries were missing information on respondant's having access to health insurance so this feature was ommited. This feature may take a very important role in someone's decision to be vaccinated. As found getting a doctor's reccomendatin greatly improved the rate of someone recieving this vaccination, therefore if more people had access to primary physicians the population that recieves recommendations will increase and in turn the vaccination rate.

---

## References

[1] <a id='h1n1_cdc_article' href='https://www.cdc.gov/flu/pandemic-resources/2009-h1n1-pandemic.html'>https://www.cdc.gov/flu/pandemic-resources/2009-h1n1-pandemic.html</a>

[2] <a id='About the National Immunization Survery' href="https://webarchive.loc.gov/all/20140511031000/http://www.cdc.gov/nchs/nis/about_nis.htm#h1n1">https://webarchive.loc.gov/all/20140511031000/http://www.cdc.gov/nchs/nis/about_nis.htm#h1n1</a>

[3] <a href='https://www.drivendata.org/competitions/66/flu-shot-learning/data/'>https://www.drivendata.org/competitions/66/flu-shot-learning/data/</a>

[4] <a id='Missing Data Imputation using Regression' href='https://www.kaggle.com/code/shashankasubrahmanya/missing-data-imputation-using-regression'>https://www.kaggle.com/code/shashankasubrahmanya/missing-data-imputation-using-regression</a>